### Nota
Este código importa todos los archivos .csv que esten en la carpeta VentaVehiculosMexico y los agrupa en un solo DF, contiene los datos de Venta de Vehículos en México del 2005-2021. 

In [154]:
import pandas as pd
import glob
import geojson
import geopandas as gpd

path = "VentaVehiculosMexico" # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
countriesTable = pd.read_csv("ExportsMap/tc_pais_destino.csv")
salesDF = pd.merge(frame, countriesTable, how="left", left_on="ID_PAIS_ORIGEN", right_on="ID_PAIS_DESTINO")


In [155]:
brandTotals = salesDF.groupby("MARCA")["UNI_VEH"].sum().sort_values(ascending=False)
top = brandTotals[0:7]
years = list(range(2006, 2021))
topSalesDF = salesDF.loc[salesDF["MARCA"].isin(top.keys())]
yearsTopSalesDF = topSalesDF.loc[topSalesDF["ANIO"].isin(years)]
salesCountryYear= yearsTopSalesDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
salesCountryYear.to_csv("ExportsMap/brandSales.csv", index=True)

In [166]:
path1 = "VehiculosExportados" # use your path
all_files1 = glob.glob(path1 + "/*.csv")
li1 = []
for filename in all_files1:
    df = pd.read_csv(filename, index_col=None, header=0)
    li1.append(df)
frame1 = pd.concat(li1, axis=0, ignore_index=True)
countriesTable = pd.read_csv("ExportsMap/tc_pais_destino.csv")
exportsDF = pd.merge(frame1, countriesTable, how="left", left_on="ID_PAIS_DESTINO", right_on="ID_PAIS_DESTINO")

brandTotals = exportsDF.groupby("MARCA")["UNI_VEH"].sum().sort_values(ascending=False)
top = brandTotals[0:7]
years = list(range(2006, 2021))
topExportsDF = exportsDF.loc[exportsDF["MARCA"].isin(top.keys())]
yearsTopExportsDF = topExportsDF.loc[topExportsDF["ANIO"].isin(years)]
exportsCountryYear= yearsTopExportsDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
exportsCountryYear.to_csv("ExportsMap/brandexports.csv", index=True)

path2 = "VentaVehiculosMexico" # use your path
all_files2 = glob.glob(path2 + "/*.csv")
li2 = []
for filename in all_files2:
    df = pd.read_csv(filename, index_col=None, header=0)
    li2.append(df)
frame2 = pd.concat(li2, axis=0, ignore_index=True)
salesDF = pd.merge(frame2, countriesTable, how="left", left_on="ID_PAIS_ORIGEN", right_on="ID_PAIS_DESTINO")

topSalesDF = salesDF.loc[salesDF["MARCA"].isin(top.keys())]
yearsTopSalesDF = topSalesDF.loc[topSalesDF["ANIO"].isin(years)]
salesCountryYear = yearsTopSalesDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
salesCountryYear.to_csv("ExportsMap/brandsales.csv", index=True)

balanceCountryYear = exportsCountryYear - salesCountryYear
balanceCountryYear.to_csv("ExportsMap/brandbalance.csv", index=True)

In [162]:
balanceCountryYear = exportsCountryYear - salesCountryYear
balanceCountryYear

ANIO  MARCA         
2006  Chrysler          175022
      Ford Motor        125185
      General Motors    167717
      Honda             -32364
      Nissan            -19495
                         ...  
2020  General Motors    531961
      Honda              71409
      Nissan            146701
      Toyota             84854
      Volkswagen        177718
Name: UNI_VEH, Length: 105, dtype: int64

In [126]:
salesCountryYear= salesDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum().unstack()
salesBrandYear = salesCountryYear.fillna(0).rename(columns={
    "ANIO": "Year"
})
totalBrands = salesDF.groupby("MARCA")["UNI_VEH"].sum().sort_values(ascending=False)
salesBrandYearTOP = salesBrandYear[totalBrands.keys()]
others = totalBrands[9:].sum()
top = totalBrands[0:9]
top["Others"] = others
salesBrandYearTOP["Others"] = salesBrandYear.iloc[9:].sum(axis=1)
salesBrandYearFinal = salesBrandYearTOP[top.keys()]

/Users/adrianmarceloflorespedraza/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [127]:
salesFinal = salesBrandYearFinal.reset_index().rename_axis("", axis="columns").rename(columns={"ANIO":"Year"})
salesFinal

,Year,Nissan,General Motors,Volkswagen,Ford Motor,Chrysler,Toyota,Honda,Mazda,KIA,Others
0,2005,234932.0,249842.0,149081.0,190839.0,124353.0,35318.0,38321.0,771.0,0.0,NaN
1,2006,228315.0,245090.0,135027.0,177595.0,128446.0,60088.0,47471.0,7495.0,0.0,NaN
2,2007,214121.0,230402.0,133240.0,147747.0,128541.0,66208.0,52951.0,16604.0,0.0,NaN
3,2008,212022.0,212378.0,119400.0,125516.0,116137.0,63306.0,54132.0,21997.0,0.0,NaN
4,2009,156186.0,138482.0,97506.0,88692.0,82337.0,51991.0,35245.0,18914.0,0.0,NaN
5,2010,189518.0,155590.0,110332.0,86735.0,78574.0,46769.0,37990.0,25116.0,0.0,NaN
6,2011,224509.0,168503.0,126831.0,85988.0,82072.0,48589.0,34426.0,29860.0,0.0,NaN
7,2012,244962.0,186383.0,133964.0,82328.0,88023.0,56278.0,52351.0,25424.0,0.0,NaN
8,2013,263477.0,201604.0,156313.0,85721.0,78974.0,60740.0,58381.0,33348.0,0.0,NaN
9,2014,291729.0,216958.0,160088.0,79097.0,69013.0,69597.0,60128.0,40997.0,0.0,1136965.0


In [78]:
salesFinal.to_csv("ExportsMap/brandSales.csv", index=False)